In [1]:
import sys
sys.path.append("../sketchformer")
sys.path.append("../src")

import tensorflow as tf
import numpy as np
import random
import pickle
from pprint import pprint

In [2]:
with open('../outputs/sample.pkl', 'rb') as f:
    sample = pickle.load(f)
print(len(sample), len(sample[0]), len(sample[0][0]['sketch']))
pprint(sample[0])

16 22 56
[{'label': 'tree',
  'position': [715, 390],
  'sketch': [[-4, 52, 0],
             [-30, 88, 0],
             [-28, 54, 1],
             [143, -175, 0],
             [-3, 5, 0],
             [-7, 71, 0],
             [0, 31, 0],
             [3, 15, 0],
             [12, 29, 0],
             [42, 46, 1],
             [-123, -206, 0],
             [-32, -1, 0],
             [-27, -5, 0],
             [-10, -3, 0],
             [-16, -11, 0],
             [-11, -23, 0],
             [0, -48, 0],
             [24, -29, 0],
             [10, -7, 0],
             [17, -5, 0],
             [7, 7, 0],
             [-1, 26, 0],
             [-5, 4, 0],
             [-5, -10, 0],
             [-2, -31, 0],
             [8, -20, 0],
             [15, -24, 0],
             [10, -9, 0],
             [14, -5, 0],
             [21, 1, 0],
             [5, 3, 0],
             [8, 16, 0],
             [0, 18, 0],
             [13, -29, 0],
             [7, -6, 0],
             [14, -9, 0],
 

In [3]:
from basic_usage.sketchformer import continuous_embeddings
sketchformer = continuous_embeddings.get_pretrained_model()

[run-experiment] resorting checkpoint if exists
[Checkpoint] Restored, step #207536


In [4]:
def convert_dataset_to_input(dataset):
    input = []
    for scene in dataset:
        sketches = list(map(lambda o: o['sketch'], scene))
        sketch_embeddings = sketchformer.get_embeddings(sketches)
        positions = list(map(lambda o: [o['position'][0] / 750, o['position'][1] / 750], scene))
        for i, (s, p) in enumerate(zip(sketch_embeddings, positions)):
            is_start = tf.convert_to_tensor([1] if i == 0 else [0], tf.float32)
            p = tf.convert_to_tensor(p, tf.float32)
            input.append(tf.concat([is_start, s, p], 0))
    return input

x = convert_dataset_to_input(sample)

[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[extract-embeddings] batch_x shape (22, 200, 5)
[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[extract-embeddings] batch_x shape (9, 200, 5)
[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[extract-embeddings] batch_x shape (11, 200, 5)
[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[extract-embeddings] batch_x shape (10, 200, 5)
[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[extract-embeddings] batch_x shape (7, 200, 5)
[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[extra

In [5]:
print(x)
print(x[0].shape)

[<tf.Tensor: shape=(131,), dtype=float32, numpy=
array([ 1.        ,  0.85353076,  0.05913709, -3.4428344 ,  3.0118911 ,
       -1.2572973 ,  2.6427326 ,  1.3781554 , -2.2432086 , -2.525485  ,
       -0.22026601, -1.2456449 , -0.9033654 , -2.269719  ,  3.109689  ,
       -1.5437568 ,  0.26202977, -0.2421196 ,  0.69563985,  0.6563781 ,
        1.2109543 , -2.0504405 ,  2.9639623 ,  0.46038112, -0.1428228 ,
        4.3030353 ,  0.8734711 ,  0.59266233, -0.85455334, -1.9457614 ,
        1.2851499 ,  1.4017317 , -2.7278843 ,  0.41635364, -0.7613081 ,
        1.4074985 , -0.13625167,  4.269829  ,  0.5241999 , -2.2732463 ,
       -1.7748095 , -0.27609712, -2.5992932 , -1.5512125 ,  1.9742644 ,
        1.4107859 ,  1.1382114 , -3.5214021 ,  1.1429511 ,  1.398088  ,
        3.0041687 , -2.542978  , -0.5440839 ,  0.39080098,  0.08334775,
        2.8255582 , -2.793876  , -0.7630989 , -1.1593372 , -1.2459538 ,
       -0.15895039,  1.773388  , -3.184871  , -0.4453405 ,  3.8020074 ,
        7.12981

In [6]:
from isketcher import InteractiveSketcher

interactive_sketcher = InteractiveSketcher(
    num_layers=2, d_model=131, num_heads=1, dff=2048, 
    target_vocab_size=8000, pe_target=6000)

# temp_target = tf.random.uniform((64, 36), dtype=tf.int64, minval=0, maxval=200)
temp_target = x

fn_out, _ = interactive_sketcher(temp_target, training=False, look_ahead_mask=None)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)

TensorShape([230, 230, 8000])